In [1]:
import pandas as pd
import json
import nltk
from nltk.probability import FreqDist
from funciones import get_sent_tokens,get_lemma,get_stems,remove_stop_words,get_word_tokens, remove_special_characters

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Alvaro\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alvaro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Alvaro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
data2 = pd.read_csv("../data/booksummaries.txt", delimiter="\t", header=None)
data2.columns = ["id","random", "book_name", "author", "date_of_publish", "genres", "summary"]
data2

,id,random,book_name,author,date_of_publish,genres,summary
0,620,/m/0hhy,Animal Farm,George Orwell,1945-08-17,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca..."
1,843,/m/0k36,A Clockwork Orange,Anthony Burgess,1962,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan..."
2,986,/m/0ldx,The Plague,Albert Camus,1947,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...
3,1756,/m/0sww,An Enquiry Concerning Human Understanding,David Hume,NaN,NaN,The argument of the Enquiry proceeds by a ser...
4,2080,/m/0wkt,A Fire Upon the Deep,Vernor Vinge,NaN,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...",The novel posits that space around the Milky ...
...,...,...,...,...,...,...,...
16554,36934824,/m/0m0p0hr,Under Wildwood,Colin Meloy,2012-09-25,NaN,"Prue McKeel, having rescued her brother from ..."
16555,37054020,/m/04f1nbs,Transfer of Power,Vince Flynn,2000-06-01,"{""/m/01jfsb"": ""Thriller"", ""/m/02xlf"": ""Fiction""}",The reader first meets Rapp while he is doing...
16556,37122323,/m/0n5236t,Decoded,Jay-Z,2010-11-16,"{""/m/0xdf"": ""Autobiography""}",The book follows very rough chronological ord...
16557,37132319,/m/0n4bqb1,America Again: Re-becoming The Greatness We Ne...,Stephen Colbert,2012-10-02,NaN,Colbert addresses topics including Wall Stree...


In [3]:
#necesitamos que esté el género y quitamos columnas que no son útiles
data2 = data2.dropna(subset="genres").drop(columns=["id", "random", 'book_name', 'author', 'date_of_publish'])

In [4]:
def fix_genre(row):
    return json.loads(row.genres).values()

data2.loc[:,"genres"] = data2.apply(fix_genre, axis=1)
data2

,genres,summary
0,"(Roman à clef, Satire, Children's literature, ...","Old Major, the old boar on the Manor Farm, ca..."
1,"(Science Fiction, Novella, Speculative fiction...","Alex, a teenager living in near-future Englan..."
2,"(Existentialism, Fiction, Absurdist fiction, N...",The text of The Plague is divided into five p...
4,"(Hard science fiction, Science Fiction, Specul...",The novel posits that space around the Milky ...
5,"(War novel, Roman à clef)","The book tells the story of Paul Bäumer, a Ge..."
...,...,...
16549,(Science Fiction),The story starts with former government agent...
16551,"(Thriller, Fiction, Suspense)",The series follows the character of Nick Ston...
16555,"(Thriller, Fiction)",The reader first meets Rapp while he is doing...
16556,(Autobiography),The book follows very rough chronological ord...


In [10]:
# 'Crime Fiction'
# 'Fantasy'
# 'Historical novel'
# 'Horror'
# 'Science Fiction'
# 'Thriller'
# 

In [17]:
initial_genres = list(data2.explode("genres").genres.value_counts().index)

In [20]:
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)

In [21]:
data2.explode("genres").genres.value_counts()

genres
Fiction                                     4747
Speculative fiction                         4314
Science Fiction                             2870
Novel                                       2463
Fantasy                                     2413
Children's literature                       2122
Mystery                                     1396
Young adult literature                       825
Suspense                                     765
Crime Fiction                                753
Historical novel                             654
Thriller                                     568
Horror                                       511
Romance novel                                435
Historical fiction                           388
Detective fiction                            341
Adventure novel                              330
Non-fiction                                  230
Alternate history                            226
Spy fiction                                  190
Comedy       

In [22]:
# 'Historical novel'
# 'Horror'
# 'Science Fiction'
# 'Thriller'
# 
def map_genres(row):
    if "science fiction" in row.lower():
        return "Science Fiction"
    if "fantasy" in row.lower():
        return "Fantasy"
    if "historical novel" in row.lower():
        return "Historical Novel"
    if "horror" in row.lower():
        return "Horror"
    if "thriller" in row.lower():
        return "Thriller"
    
    if "guide" in row.lower():
        return "Guide"
    if "western" in row.lower():
        return "Western"
    if "romance" in row.lower():
        return "Romance"
    if "fiction" in row.lower():
        return "Fiction"
    if "comedy" in row.lower():
        return "Comedy"
    if "pornography" in row.lower():
        return "Pornography"
    if "history" in row.lower():
        return "History"
    if "novel" in row.lower():
        return "Novel"
    if "poetry" in row.lower():
        return "Poetry"
    if "alien" in row.lower():
        return "Science Fiction"
    if "cyber" in row.lower():
        return "Science Fiction"
    if "punk" in row.lower():
        return "Fiction"
    
    if "computer science" in row.lower():
        return "Textbook"
    if "literary theory" in row.lower():
        return "Textbook"
    if "computer science" in row.lower():
        return "Textbook"
    if "computer science" in row.lower():
        return "Textbook"
    if "neuroscience" in row.lower():
        return "Textbook"
    
    return "REMOVE"
    

In [25]:
data2.explode("genres").genres.apply(map_genres).value_counts()

genres
Fiction             11352
REMOVE               6869
Novel                3292
Science Fiction      2972
Fantasy              2594
Historical Novel      654
Thriller              627
Romance               535
Horror                511
History               302
Comedy                196
Western                65
Poetry                 30
Textbook                3
Pornography             1
Guide                   1
Name: count, dtype: int64